In [ ]:
#default_exp foundation

In [ ]:
#export
from fastcore.imports import *

In [ ]:
from fastcore.test import *
from nbdev.showdoc import *

# Core

> Basic functions used in the fastai library

In [ ]:
# export
defaults = SimpleNamespace()

## Metaclasses

See this [blog post](https://realpython.com/python-metaclasses/) for more information about metaclasses. 

- `FixSigMeta` preserves information that enables [intropsection of signatures](https://www.python.org/dev/peps/pep-0362/#:~:text=Python%20has%20always%20supported%20powerful,fully%20reconstruct%20the%20function's%20signature.) (i.e. tab completion in IDEs) when certain types of inheritence would otherwise obfuscate this introspection.
- `PrePostInitMeta` ensures that the classes defined with it run `__pre_init__` and `__post_init__` (without having to write `self.__pre_init__()` and `self.__post_init__()`  in the actual `init`
- `NewChkMeta` gives the `PrePostInitMeta` functionality and ensures classes defined with it don't re-create an object of their type whenever it's passed to the constructor
- `BypassNewMeta` ensures classes defined with it can easily be casted form objects they subclass.

In [ ]:
#export 
def _rm_self(sig):
    sigd = dict(sig.parameters)
    sigd.pop('self')
    return sig.replace(parameters=sigd.values())

In [ ]:
#export
class FixSigMeta(type):
    "A metaclass that fixes the signature on classes that override `__new__`"
    def __new__(cls, name, bases, dict):
        res = super().__new__(cls, name, bases, dict)
        if res.__init__ is not object.__init__: res.__signature__ = _rm_self(inspect.signature(res.__init__))
        return res

In [ ]:
show_doc(FixSigMeta, title_level=3)

<h3 id="FixSigMeta" class="doc_header"><code>class</code> <code>FixSigMeta</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>FixSigMeta</code>(**`name`**, **`bases`**, **`dict`**) :: `type`

A metaclass that fixes the signature on classes that override `__new__`

When you inherit from a class that defines `__new__`, or a metaclass that defines `__call__`, the signature of your `__init__` method is obfuscated such that tab completion no longer works.  `FixSigMeta` fixes this issue and restores signatures.

To understand what `FixSigMeta` does, it is useful to inspect an object's signature.  You can inspect the signature of an object with `inspect.signature`:

In [ ]:
class T:
    def __init__(self, a, b, c): pass
    
inspect.signature(T)

<Signature (a, b, c)>

This corresponds to tab completion working in the normal way:

<img alt="Tab completion in a Jupyter Notebook." caption="" src="images/att_00005.png">

However, when you inherhit from a class that defines `__new__` or a metaclass that defines `__call__` this obfuscates the signature by overriding your class with the signature of `__new__`, which prevents tab completion from displaying useful information:

In [ ]:
class Foo:
    def __new__(self, **args): pass

class Bar(Foo):
    def __init__(self, d, e, f): pass
    
inspect.signature(Bar)

<Signature (**args)>

<img alt="Tab completion in a Jupyter Notebook." caption="" src="images/att_00006.png">

Finally, the signature and tab completion can be restored by inheriting from the metaclass `FixSigMeta` as shown below:

In [ ]:
class Bar(Foo, metaclass=FixSigMeta):
    def __init__(self, d, e, f): pass
    
test_sig(Bar, '(d, e, f)')
inspect.signature(Bar)

<Signature (d, e, f)>

<img alt="Tab completion in a Jupyter Notebook." caption="" src="images/att_00007.png">

If you need to define a metaclass that overrides `__call__` (as done in `PrePostInitMeta`), you need to inherit from `FixSigMeta` instead of `type` when constructing the metaclass to preserve the signature in `__init__`.  Be careful not to override `__new__` when doing this:

In [ ]:
class TestMeta(FixSigMeta):
    # __new__ comes from FixSigMeta
    def __call__(cls, *args, **kwargs): pass
    
class T(metaclass=TestMeta):
    def __init__(self, a, b): pass
    
test_sig(T, '(a, b)')

On the other hand, if you fail to inherit from `FixSigMeta` when inheriting from a metaclass that overrides `__call__`, your signature will reflect that of `__call__` instead (which is often undesirable):

In [ ]:
class GenericMeta(type):
    "A boilerplate metaclass that doesn't do anything for testing."
    def __new__(cls, name, bases, dict):
        return super().__new__(cls, name, bases, dict)
    def __call__(cls, *args, **kwargs): pass

class T2(metaclass=GenericMeta):
    def __init__(self, a, b): pass

# We can avoid this by inheriting from the metaclass `FixSigMeta`
test_sig(T2, '(*args, **kwargs)')

In [ ]:
#export
class PrePostInitMeta(FixSigMeta):
    "A metaclass that calls optional `__pre_init__` and `__post_init__` methods"
    def __call__(cls, *args, **kwargs):
        res = cls.__new__(cls)
        if type(res)==cls:
            if hasattr(res,'__pre_init__'): res.__pre_init__(*args,**kwargs)
            res.__init__(*args,**kwargs)
            if hasattr(res,'__post_init__'): res.__post_init__(*args,**kwargs)
        return res

In [ ]:
show_doc(PrePostInitMeta, title_level=3)

<h3 id="PrePostInitMeta" class="doc_header"><code>class</code> <code>PrePostInitMeta</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>PrePostInitMeta</code>(**`name`**, **`bases`**, **`dict`**) :: [`FixSigMeta`](/foundation.html#FixSigMeta)

A metaclass that calls optional `__pre_init__` and `__post_init__` methods

`__pre_init__`  and `__post_init__` are useful for intializing variables or performing tasks prior to or after `__init__` being called, respectively.  Fore example:

In [ ]:
class _T(metaclass=PrePostInitMeta):
    def __pre_init__(self):  self.a  = 0; 
    def __init__(self,b=0):  self.b = self.a + 1; assert self.b==1
    def __post_init__(self): self.c = self.b + 2; assert self.c==3

t = _T()
test_eq(t.a, 0) # set with __pre_init__
test_eq(t.b, 1) # set with __init__
test_eq(t.c, 3) # set with __post_init__

One excellent use for `PrePostInitMeta` is avoiding the `__super__().__init__()` boilerplate associated with subclassing. For example, to build models with pytorch, [it is customary to subclass nn.module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module).  You can use `ProPostInitMeta` to avoid this like so:

```py
from torch import nn
class Module(nn.Module, metaclass=PrePostInitMeta):
    "Same as `nn.Module`, but no need for subclasses to call `super().__init__`"
    def __pre_init__(self, *args, **kwargs): super().__init__()
    def __init__(self): pass
```

Below is a toy example of how this works:

In [ ]:
class TestParent():
    def __init__(self): self.h = 10
        
class TestChild(TestParent, metaclass=PrePostInitMeta):
    def __pre_init__(self, *args, **kwargs): super().__init__()
    def __init__(self): self.k = self.h + 2
    
t = TestChild()
test_eq(t.h, 10) # h=10 is initialized in the parent class
test_eq(t.k, 12)

In [ ]:
#export
class NewChkMeta(FixSigMeta):
    "Metaclass to avoid recreating object passed to constructor"
    def __call__(cls, x=None, *args, **kwargs):
        if not args and not kwargs and x is not None and isinstance(x,cls):
            x._newchk = 1
            return x

        res = super().__call__(*((x,) + args), **kwargs)
        res._newchk = 0
        return res

In [ ]:
show_doc(NewChkMeta, title_level=3)

<h3 id="NewChkMeta" class="doc_header"><code>class</code> <code>NewChkMeta</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>NewChkMeta</code>(**`name`**, **`bases`**, **`dict`**) :: [`FixSigMeta`](/foundation.html#FixSigMeta)

Metaclass to avoid recreating object passed to constructor

`NewChkMeta` is used when an object of the same type is the first argument to your class's constructor (i.e. the `__init__` function), and you would rather it not create a new object but point to the same exact object.  

This is used in `L`, for example, to avoid creating a new object when the object is already of type `L`.  This allows the users to defenisvely instantiate an `L` object and just return a reference to the same object if it already happens to be of type `L`.

For example, the below class `_T` **optionally** accepts an object `o` as its first argument.  A new object is returned upon instantiation per usual:

In [ ]:
class _T():
    "Testing"
    def __init__(self, o): 
        # if `o` is not an object without an attribute `foo`, set foo = 1
        self.foo = getattr(o,'foo',1)

In [ ]:
t = _T(3)
test_eq(t.foo,1) # 1 was not of type _T, so foo = 1

t2 = _T(t) #t1 is of type _T
assert t is not t2 # t1 and t2 are different objects

However, if we want `_T` to return a reference to the same object when passed an an ojbect of type `_T` we can inherit from the `NewChkMeta` class as illustrated below:

In [ ]:
class _T(metaclass=NewChkMeta):
    "Testing with metaclass NewChkMeta"
    def __init__(self, o=None, b=1):
        # if `o` is not an object without an attribute `foo`, set foo = 1
        self.foo = getattr(o,'foo',1)
        self.b = b

We can now test `t` and `t2` are now pointing at the same object when using this new definition of `_T`:

In [ ]:
t = _T(3)
test_eq(t.foo,1) # 1 was not of type _T, so foo = 1

t2 = _T(t) # t2 will now reference t

test_is(t, t2) # t and t2 are the same object
t2.foo = 5 # this will also change t.foo to 5 because it is the same object
test_eq(t.foo, 5)
test_eq(t2.foo, 5)

However, there is one exception to how `NewChkMeta` works.  **If you pass any additional arguments in the constructor a new object is returned**, even if the first object is of the same type.  For example, consider the below example where we pass the additional argument `b` into the constructor:

In [ ]:
t3 = _T(t, b=1)
assert t3 is not t

t4 = _T(t) # without any arguments the constructor will return a reference to the same object
assert t4 is t

Finally, it should be noted that `NewChkMeta` as well as all other metaclases in this section, inherit from `FixSigMeta`.  This means class signatures will always be preserved when inheriting from this metaclass (see docs for `FixSigMeta` for more details):

In [ ]:
test_sig(_T, '(o=None, b=1)')

In [ ]:
#export
class BypassNewMeta(FixSigMeta):
    "Metaclass: casts `x` to this class if it's of type `cls._bypass_type`"
    def __call__(cls, x=None, *args, **kwargs):
        if hasattr(cls, '_new_meta'): x = cls._new_meta(x, *args, **kwargs)
        elif not isinstance(x,getattr(cls,'_bypass_type',object)) or len(args) or len(kwargs):
            x = super().__call__(*((x,)+args), **kwargs)
        if cls!=x.__class__: x.__class__ = cls
        return x

In [ ]:
show_doc(BypassNewMeta, title_level=3)

<h3 id="BypassNewMeta" class="doc_header"><code>class</code> <code>BypassNewMeta</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>BypassNewMeta</code>(**`name`**, **`bases`**, **`dict`**) :: [`FixSigMeta`](/foundation.html#FixSigMeta)

Metaclass: casts `x` to this class if it's of type `cls._bypass_type`

`BypassNewMeta` is identical to `NewChkMeta`, except for checking for a class as the same type, we instead check for a class of type specified in attribute `_bypass_type`.

In NewChkMeta, objects of the same type passed to the constructor (without arguments) would result into a new variable referencing the same object.  However, with `BypassNewMeta` this only occurs if the type matches the `_bypass_type` of the class you are defining:

In [ ]:
class _TestA: pass
class _TestB: pass

class _T(_TestA, metaclass=BypassNewMeta):
    _bypass_type=_TestB
    def __init__(self,x): self.x=x

In the below example, `t` does not refer to `t2` because `t` is of type `_TestA` while `_T._bypass_type` is of type `TestB`:

In [ ]:
t = _TestA()
t2 = _T(t)
assert t is not t2

However, if `t` is set to `_TestB` to match `_T._bypass_type`, then both `t` and `t2` will refer to the same object.

In [ ]:
t = _TestB()
t2 = _T(t)
t2.new_attr = 15

test_is(t, t2)
# since t2 just references t these will be the same
test_eq(t.new_attr, t2.new_attr)

# likewise, chaning an attribute on t will also affect t2 because they both point to the same object.
t.new_attr = 9
test_eq(t2.new_attr, 9)

## Foundational Functions

In [ ]:
#export
def copy_func(f):
    "Copy a non-builtin function (NB `copy.copy` does not work for this)"
    if not isinstance(f,FunctionType): return copy(f)
    fn = FunctionType(f.__code__, f.__globals__, f.__name__, f.__defaults__, f.__closure__)
    fn.__dict__.update(f.__dict__)
    return fn

Sometimes it may be desirable to make a copy of a function that doesn't point to the original object.  When you use Python's built in `copy.copy` or `copy.deepcopy` to copy a function, you get a reference to the original object:

In [ ]:
import copy as cp
def foo(): pass
a = cp.copy(foo)
b = cp.deepcopy(foo)

a.someattr = 'hello' # since a and b point at the same object, updating a will update b
test_eq(b.someattr, 'hello')

assert a is foo and b is foo

However, with `copy_func`, you can retrieve a copy of a function without a reference to the original object:

In [ ]:
c = copy_func(foo) # c is an indpendent object
assert c is not foo

In [ ]:
#export
def patch_to(cls, as_prop=False, cls_method=False):
    "Decorator: add `f` to `cls`"
    if not isinstance(cls, (tuple,list)): cls=(cls,)
    def _inner(f):
        for c_ in cls:
            nf = copy_func(f)
            # `functools.update_wrapper` when passing patched function to `Pipeline`, so we do it manually
            for o in functools.WRAPPER_ASSIGNMENTS: setattr(nf, o, getattr(f,o))
            nf.__qualname__ = f"{c_.__name__}.{f.__name__}"
            if cls_method:
                setattr(c_, f.__name__, MethodType(nf, c_))
            else:
                setattr(c_, f.__name__, property(nf) if as_prop else nf)
        return f
    return _inner

The `@patch_to` decorator allows you to [monkey patch](https://stackoverflow.com/questions/5626193/what-is-monkey-patching) a function into a class as a method:

In [ ]:
class _T3(int): pass  

@patch_to(_T3)
def func1(self, a): return self+a

t = _T3(1) # we initilized `t` to a type int = 1
test_eq(t.func1(2), 3) # we add 2 to `t`, so 2 + 1 = 3

You can access instance properties in the usual way via `self`:

In [ ]:
class _T4():
    def __init__(self, g): self.g = g
        
@patch_to(_T4)
def greet(self, x): return self.g + x
        
t = _T4('hello ') # this sets self.g = 'helllo '
test_eq(t.greet('world'), 'hello world') #t.greet('world') will append 'world' to 'hello '

You can instead specify that the method should be a class method by setting `cls_method=True`:

In [ ]:
class _T5(int): attr = 3 # attr is a class attribute we will access in a later method
    
@patch_to(_T5, cls_method=True)
def func(cls, x): return cls.attr + x # you can access class attributes in the normal way

t = _T5()
test_eq(t.func(4), 7)

Additionally you can specify that the function you want to patch should be a class attribute with `as_prop` = False

In [ ]:
@patch_to(_T5, as_prop=True)
def add_ten(self): return self + 10

t = _T5(4)
test_eq(t.add_ten, 14)

Instead of passing one class to the `@patch_to` decorator, you can pass multiple classes in a tuple to simulteanously patch more than one class with the same method:

In [ ]:
class _T6(int): pass
class _T7(int): pass

@patch_to((_T6,_T7))
def func_mult(self, a): return self*a

t = _T6(2)
test_eq(t.func_mult(4), 8)
t = _T7(2)
test_eq(t.func_mult(4), 8)

In [ ]:
#export
def patch(f):
    "Decorator: add `f` to the first parameter's class (based on f's type annotations)"
    cls = next(iter(f.__annotations__.values()))
    return patch_to(cls)(f)

`@patch` is an alternative to `@patch_to` that allows you similarly monkey patch class(es) by using [type annotations](https://docs.python.org/3/library/typing.html):

In [ ]:
class _T8(int): pass  

@patch
def func(self:_T8, a): return self+a

t = _T8(1)  # we initilized `t` to a type int = 1
test_eq(t.func(3), 4) # we add 3 to `t`, so 3 + 1 = 4
test_eq(t.func.__qualname__, '_T8.func')

Similarly to `patch_to`, you can supply a tuple of classes instead of a single class in your type annotations to patch multiple classes:

In [ ]:
class _T9(int): pass 

@patch
def func2(x:(_T8,_T9), a): return x*a # will patch both _T8 and _T9

t = _T8(2)
test_eq(t.func2(4), 8)
test_eq(t.func2.__qualname__, '_T8.func2')

t = _T9(2)
test_eq(t.func2(4), 8)
test_eq(t.func2.__qualname__, '_T9.func2')

In [ ]:
#export
def patch_property(f):
    "Decorator: add `f` as a property to the first parameter's class (based on f's type annotations)"
    cls = next(iter(f.__annotations__.values()))
    return patch_to(cls, as_prop=True)(f)

This is similar to using `patch_to` with the argument `as_prop=True`.

In [ ]:
class _T3(int): pass

@patch_property
def prop(self:_T3): return self+1 #this will patch _T3 because of self's type annotation

t = _T3(1)
test_eq(t.prop, 2)

You may also patch multiple classes at once by passing a tuple of objects as the type annotation for the first argument of your function:

In [ ]:
class _T4(int): pass

@patch_property
def newprop(self:(_T3, _T4)): return self+5

t = _T3(1)
test_eq(t.newprop, 6)
t = _T4(1)
test_eq(t.newprop, 6)

In [ ]:
#export
def _mk_param(n,d=None): return inspect.Parameter(n, inspect.Parameter.KEYWORD_ONLY, default=d)

In [ ]:
#export
def use_kwargs_dict(keep=False, **kwargs):
    "Decorator: replace `**kwargs` in signature with `names` params"
    def _f(f):
        sig = inspect.signature(f)
        sigd = dict(sig.parameters)
        k = sigd.pop('kwargs')
        s2 = {n:_mk_param(n,d) for n,d in kwargs.items() if n not in sigd}
        sigd.update(s2)
        if keep: sigd['kwargs'] = k
        f.__signature__ = sig.replace(parameters=sigd.values())
        return f
    return _f

Replace all `**kwargs` with named arguments like so:

In [ ]:
@use_kwargs_dict(y=1,z=None)
def foo(a, b=1, **kwargs): pass

test_sig(foo, '(a, b=1, *, y=1, z=None)')

Add named arguments, but optionally keep `**kwargs` by setting `keep=True`:

In [ ]:
@use_kwargs_dict(y=1,z=None, keep=True)
def foo(a, b=1, **kwargs): pass

test_sig(foo, '(a, b=1, *, y=1, z=None, **kwargs)')

In [ ]:
#export
def use_kwargs(names, keep=False):
    "Decorator: replace `**kwargs` in signature with `names` params"
    def _f(f):
        sig = inspect.signature(f)
        sigd = dict(sig.parameters)
        k = sigd.pop('kwargs')
        s2 = {n:_mk_param(n) for n in names if n not in sigd}
        sigd.update(s2)
        if keep: sigd['kwargs'] = k
        f.__signature__ = sig.replace(parameters=sigd.values())
        return f
    return _f

`use_kwargs` is different than `use_kwargs_dict` as it only replaces `**kwargs` with named parameters without any default values:

In [ ]:
@use_kwargs(['y', 'z'])
def foo(a, b=1, **kwargs): pass

test_sig(foo, '(a, b=1, *, y=None, z=None)')

You may optionally keep the `**kwargs` argument in your signature by setting `keep=True`:

In [ ]:
@use_kwargs(['y', 'z'], keep=True)
def foo(a, *args, b=1, **kwargs): pass
test_sig(foo, '(a, *args, b=1, y=None, z=None, **kwargs)')

In [ ]:
# export
def delegates(to=None, keep=False, but=None):
    "Decorator: replace `**kwargs` in signature with params from `to`"
    if but is None: but = []
    def _f(f):
        if to is None: to_f,from_f = f.__base__.__init__,f.__init__
        else:          to_f,from_f = to,f
        from_f = getattr(from_f,'__func__',from_f)
        to_f = getattr(to_f,'__func__',to_f)
        if hasattr(from_f,'__delwrap__'): return f
        sig = inspect.signature(from_f)
        sigd = dict(sig.parameters)
        k = sigd.pop('kwargs')
        s2 = {k:v for k,v in inspect.signature(to_f).parameters.items()
              if v.default != inspect.Parameter.empty and k not in sigd and k not in but}
        sigd.update(s2)
        if keep: sigd['kwargs'] = k
        else: from_f.__delwrap__ = to_f
        from_f.__signature__ = sig.replace(parameters=sigd.values())
        return f
    return _f

A common Python idiom is to accept `**kwargs` in addition to named parameters that are passed onto other function calls. It is especially common to use `**kwargs` when you want to give the user an option to override default parameters of any functions or methods being called by the parent function.

For example, suppose we have have a function `foo` that passes arguments to `baz` like so:

In [ ]:
def baz(a, b=2, c =3): return a + b + c

def foo(c, a, **kwargs):
    return c + baz(a, **kwargs)

assert foo(c=1, a=1) == 7

The problem with this approach is the api for `foo` is obfuscated. Users cannot introspect what the valid arguments for `**kwargs` are without reading the source code.  When a user tries tries to introspect the signature of `foo`, they are presented with this:

In [ ]:
inspect.signature(foo)

<Signature (c, a, **kwargs)>

We can address this issue by using the decorator `delegates` to include parameters from other functions.  For example, if we apply the `delegates` decorator to `foo` to include parameters from `baz`:

In [ ]:
@delegates(baz)
def foo(c, a, **kwargs):
    return c + baz(a, **kwargs)

test_sig(foo, '(c, a, b=2)')
inspect.signature(foo)

<Signature (c, a, b=2)>

We can optionally decide to keep `**kwargs` by setting `keep=True`:

In [ ]:
@delegates(baz, keep=True)
def foo(c, a, **kwargs):
    return c + baz(a, **kwargs)

test_sig(foo, '(c, a, b=2, **kwargs)')
inspect.signature(foo)

<Signature (c, a, b=2, **kwargs)>

It is important to note that **only parameters with default parameters are included**.  For example, in the below scenario only `c`, but NOT `e` and `d` are included in the signature of `foo` after applying `delegates`:

In [ ]:
def basefoo(e, d, c=2): pass

@delegates(basefoo)
def foo(a, b=1, **kwargs): pass
test_sig(foo, '(a, b=1, c=2)') # e and d are not included b/c they don't have default parameters.
inspect.signature(foo)

<Signature (a, b=1, c=2)>

The reason that required arguments (i.e. those without default parameters) are automatically excluded is that you should be explicitly implementing required arguments into your function's signature rather than relying on `delegates`.

Additionally, you can exclude specific parameters from being included in the signature with the  `but` parameter.  In the example below, we exclude the parameter `d`:

In [ ]:
def basefoo(e, c=2, d=3): pass

@delegates(basefoo, but= ['d'])
def foo(a, b=1, **kwargs): pass

test_sig(foo, '(a, b=1, c=2)')
inspect.signature(foo)

<Signature (a, b=1, c=2)>

You can also use `delegates` between methods in a class.  Here is an example of `delegates` with class methods:

In [ ]:
# example 1: class methods
class _T():
    @classmethod
    def foo(cls, a=1, b=2):
        pass
    
    @classmethod
    @delegates(foo)
    def bar(cls, c=3, **kwargs):
        pass

test_sig(_T.bar, '(c=3, a=1, b=2)')

Here is the same example with instance methods:

In [ ]:
# example 2: instance methods
class _T():
    def foo(self, a=1, b=2):
        pass
    
    @delegates(foo)
    def bar(self, c=3, **kwargs):
        pass

t = _T()
test_sig(t.bar, '(c=3, a=1, b=2)')

You can also delegate between classes.  By default, the `delegates` decorator will delegate to the superclass:

In [ ]:
class BaseFoo:
    def __init__(self, e, c=2): pass

@delegates()# since no argument was passsed here we delegate to the superclass
class Foo(BaseFoo):
    def __init__(self, a, b=1, **kwargs): super().__init__(**kwargs)

test_sig(Foo, '(a, b=1, c=2)')

In [ ]:
#export
def method(f):
    "Mark `f` as a method"
    # `1` is a dummy instance since Py3 doesn't allow `None` any more
    return MethodType(f, 1)

In [ ]:
#export
def _funcs_kwargs(cls, as_method):
    old_init = cls.__init__
    def _init(self, *args, **kwargs):
        for k in cls._methods:
            arg = kwargs.pop(k,None)
            if arg is not None:
                if as_method: arg = method(arg)
                if isinstance(arg,MethodType): arg = MethodType(arg.__func__, self)
                setattr(self, k, arg)
        old_init(self, *args, **kwargs)
    functools.update_wrapper(_init, old_init)
    cls.__init__ = use_kwargs(cls._methods)(_init)
    if hasattr(cls, '__signature__'): cls.__signature__ = _rm_self(inspect.signature(cls.__init__))
    return cls

In [ ]:
#export
def funcs_kwargs(as_method=False):
    "Replace methods in `cls._methods` with those from `kwargs`"
    if callable(as_method): return _funcs_kwargs(as_method, False)
    return partial(_funcs_kwargs, as_method=as_method)

The `func_kwargs` decorator allows you to add a list of functions or methods to an existing class.  You must set this list as a class attribute named `_methods` when defining your class.  Additionally, you must incldue the `**kwargs` argument in the `___init__` method of your class.

After defining your class this way, you can add functions to your class upon instantation as illusrated below.

For example, we define class `T` to allow adding the function `b` to class `T` as follows (note that this function is stored as an attribute of `T` and doesn't have access to `cls` or `self`):

In [ ]:
@funcs_kwargs
class T:
    _methods=['b'] # allows you to add method b upon instantiation
    def __init__(self, f=1, **kwargs): pass # don't forget to include **kwargs in __init__
    def a(self): return 1
    def b(self): return 2
    
t = T()
test_eq(t.a(), 1)
test_eq(t.b(), 2)

Because we defined the class `T` this way, the signature of `T` indicates the option to add the function or method(s) specified in `_methods`.  In this example, `b` is added to the signature:

In [ ]:
test_sig(T, '(f=1, *, b=None)')
inspect.signature(T)

<Signature (f=1, *, b=None)>

You can now add the function `b` to class `T` upon instantiation:

In [ ]:
def _new_func(): return 5

t = T(b = _new_func)
test_eq(t.b(), 5)

If you try to add a function with a name not listed in `_methods` it will be ignored. In the below example, the attempt to add a function named `a` is ignored:

In [ ]:
t = T(a = lambda:3)
test_eq(t.a(), 1) # the attempt to add a is ignored and uses the original method instead.

Note that you can also add methods not defined in the original class as long it is specified in the `_methods` attribute:

In [ ]:
@funcs_kwargs
class T:
    _methods=['c']
    def __init__(self, f=1, **kwargs): pass

t = T(c = lambda: 4)
test_eq(t.c(), 4)

Until now, these examples showed how to add functions stored as an instance attribute without access to `self`.  However, if you need access to `self` you can set `as_method=True` in the `func_kwargs` decorator to add a method instead:

In [ ]:
def _f(self,a=1): return self.num + a # access the num attribute from the instance

@funcs_kwargs(as_method=True)
class T: 
    _methods=['b']
    num = 5
    
t = T(b = _f) # adds method b
test_eq(t.b(5), 10) # self.num + 5 = 10

Here is an example of how you might use this functionality with inheritence:

In [ ]:
def _f(self,a=1): return self.num * a #multiply instead of add 

class T2(T):
    def __init__(self,num):
        super().__init__(b = _f) # add method b from the super class
        self.num=num
        
t = T2(num=3)
test_eq(t.b(a=5), 15) # 3 * 5 = 15
test_sig(T2, '(num)')

In [ ]:
#hide
def _g(a=1): return a+1
class T3(T): b = staticmethod(_g)
t = T3()
test_eq(t.b(2), 3)

In [ ]:
#hide
#test it works with PrePostInitMeta
class A(metaclass=PrePostInitMeta): pass

@funcs_kwargs
class B(A):
    _methods = ['m1']
    def __init__(self, **kwargs): pass
    
test_sig(B, '(*, m1=None)')

Runtime type checking is handy, so let's make it easy!

In [ ]:
@contextmanager
def working_directory(path):
    "Change working directory to `path` and return to previous on exit."
    prev_cwd = Path.cwd()
    os.chdir(path)
    try: yield
    finally: os.chdir(prev_cwd)

In [ ]:
#export
def add_docs(cls, cls_doc=None, **docs):
    "Copy values from `docs` to `cls` docstrings, and confirm all public methods are documented"
    if cls_doc is not None: cls.__doc__ = cls_doc
    for k,v in docs.items():
        f = getattr(cls,k)
        if hasattr(f,'__func__'): f = f.__func__ # required for class methods
        f.__doc__ = v
    # List of public callables without docstring
    nodoc = [c for n,c in vars(cls).items() if callable(c)
             and not n.startswith('_') and c.__doc__ is None]
    assert not nodoc, f"Missing docs: {nodoc}"
    assert cls.__doc__ is not None, f"Missing class docs: {cls}"

`add_docs` allows you to add docstrings to a class and its associated methods.  This function allows you to group docstrings together seperate from your code, which enables you to define one-line functions as well as organize your code more succintly. We believe this confers a number of benefits which we discuss in [our style guide](https://docs.fast.ai/dev/style.html).

Suppose you have the following undocumented class:

In [ ]:
class T:
    def foo(self): pass
    def bar(self): pass

You can add documentation to this class like so:

In [ ]:
add_docs(T, cls_doc="A docstring for the class.",
            foo="The foo method.",
            bar="The bar method.")

Now, docstrings will appear as expected:

In [ ]:
test_eq(T.__doc__, "A docstring for the class.")
test_eq(T.foo.__doc__, "The foo method.")
test_eq(T.bar.__doc__, "The bar method.")

`add_docs` also validates that all of  your public methods contain a docstring.  If one of your methods is not documented, it will raise an error:

In [ ]:
class T:
    def foo(self): pass
    def bar(self): pass

f=lambda: add_docs(T, "A docstring for the class.", foo="The foo method.")
test_fail(f, contains="Missing docs")

In [ ]:
#hide
class _T:
    def f(self): pass
    @classmethod
    def g(cls): pass
add_docs(_T, "a", f="f", g="g")

test_eq(_T.__doc__, "a")
test_eq(_T.f.__doc__, "f")
test_eq(_T.g.__doc__, "g")

In [ ]:
#export
def docs(cls):
    "Decorator version of `add_docs`, using `_docs` dict"
    add_docs(cls, **cls._docs)
    return cls

Instead of using `add_docs`, you can use the decorator `docs` as shown below.  Note that the docstring for the class can be set with the argument `cls_doc`:

In [ ]:
@docs
class _T:
    def f(self): pass
    def g(cls): pass
    
    _docs = dict(cls_doc="The class docstring", 
                 f="The docstring for method f.",
                 g="A different docstring for method g.")

    
test_eq(_T.__doc__, "The class docstring")
test_eq(_T.f.__doc__, "The docstring for method f.")
test_eq(_T.g.__doc__, "A different docstring for method g.")

For either the `docs` decorator or the `add_docs` function, you can still define your docstrings in the normal way.  Below we set the docstring for the class as usual, but define the method docstrings through the `_docs` attribute:

In [ ]:
@docs
class _T:
    "The class docstring"
    def f(self): pass
    _docs = dict(f="The docstring for method f.")

    
test_eq(_T.__doc__, "The class docstring")
test_eq(_T.f.__doc__, "The docstring for method f.")

In [ ]:
#export
def custom_dir(c, add:list):
    "Implement custom `__dir__`, adding `add` to `cls`"
    return dir(type(c)) + list(c.__dict__.keys()) + add

In [ ]:
show_doc(is_iter)

<h4 id="is_iter" class="doc_header"><code>is_iter</code><a href="https://github.com/fastai/fastcore/tree/master/fastcore/imports.py#L37" class="source_link" style="float:right">[source]</a></h4>

> <code>is_iter</code>(**`o`**)

Test whether `o` can be used in a `for` loop

In [ ]:
assert is_iter([1])
assert not is_iter(array(1))
assert is_iter(array([1,2]))
assert (o for o in range(3))

In [ ]:
#export
class _Arg:
    def __init__(self,i): self.i = i
arg0 = _Arg(0)
arg1 = _Arg(1)
arg2 = _Arg(2)
arg3 = _Arg(3)
arg4 = _Arg(4)

In [ ]:
# export
def coll_repr(c, max_n=10):
    "String repr of up to `max_n` items of (possibly lazy) collection `c`"
    return f'(#{len(c)}) [' + ','.join(itertools.islice(map(repr,c), max_n)) + (
        '...' if len(c)>10 else '') + ']'

In [ ]:
test_eq(coll_repr(range(1000), 5), '(#1000) [0,1,2,3,4...]')

In [ ]:
# export
def mask2idxs(mask):
    "Convert bool mask or index list to index `L`"
    if isinstance(mask,slice): return mask
    mask = list(mask)
    if len(mask)==0: return []
    it = mask[0]
    if hasattr(it,'item'): it = it.item()
    if isinstance(it,(bool,NoneType,np.bool_)): return [i for i,m in enumerate(mask) if m]
    return [int(i) for i in mask]

In [ ]:
# just for tests
import torch

In [ ]:
test_eq(mask2idxs([False,True,False,True]), [1,3])
test_eq(mask2idxs(array([False,True,False,True])), [1,3])
test_eq(mask2idxs(torch.tensor([False,True,False,True])), [1,3])
test_eq(mask2idxs(array([1,2,3])), [1,2,3])

In [ ]:
#export
def _is_array(x): return hasattr(x,'__array__') or hasattr(x,'iloc')

def _listify(o):
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, str) or _is_array(o): return [o]
    if is_iter(o): return list(o)
    return [o]

In [ ]:
#export
def cycle(o):
    "Like `itertools.cycle` except creates list of `None`s if `o` is empty"
    o = _listify(o)
    return itertools.cycle(o) if o is not None and len(o) > 0 else itertools.cycle([None])

In [ ]:
test_eq(itertools.islice(cycle([1,2,3]),5), [1,2,3,1,2])
test_eq(itertools.islice(cycle([]),3), [None]*3)
test_eq(itertools.islice(cycle(None),3), [None]*3)
test_eq(itertools.islice(cycle(1),3), [1,1,1])

In [ ]:
#export
def zip_cycle(x, *args):
    "Like `itertools.zip_longest` but `cycle`s through elements of all but first argument"
    return zip(x, *map(cycle,args))

In [ ]:
test_eq(zip_cycle([1,2,3,4],list('abc')), [(1, 'a'), (2, 'b'), (3, 'c'), (4, 'a')])

In [ ]:
#export
def is_indexer(idx):
    "Test whether `idx` will index a single item in a list"
    return isinstance(idx,int) or not getattr(idx,'ndim',1)

In [ ]:
#export
def negate_func(f):
    "Create new function that negates result of `f`"
    def _f(*args, **kwargs): return not f(*args, **kwargs)
    return _f

In [ ]:
def f(a): return a>0
test_eq(f(1),True)
test_eq(negate_func(f)(1),False)
test_eq(negate_func(f)(a=-1),True)

## Attribute Delegation

In [ ]:
#export
class GetAttr:
    "Inherit from this to have all attr accesses in `self._xtra` passed down to `self.default`"
    _default='default'
    def _component_attr_filter(self,k):
        if k.startswith('__') or k in ('_xtra',self._default): return False
        xtra = getattr(self,'_xtra',None)
        return xtra is None or k in xtra
    def _dir(self): return [k for k in dir(getattr(self,self._default)) if self._component_attr_filter(k)]
    def __getattr__(self,k):
        if self._component_attr_filter(k):
            attr = getattr(self,self._default,None)
            if attr is not None: return getattr(attr,k)
        raise AttributeError(k)
    def __dir__(self): return custom_dir(self,self._dir())
#     def __getstate__(self): return self.__dict__
    def __setstate__(self,data): self.__dict__.update(data)

In [ ]:
show_doc(GetAttr, title_level=4)

<h4 id="GetAttr" class="doc_header"><code>class</code> <code>GetAttr</code><a href="" class="source_link" style="float:right">[source]</a></h4>

> <code>GetAttr</code>()

Inherit from this to have all attr accesses in `self._xtra` passed down to `self.default`

Inherit from `GetAttr` to have attr access passed down to an instance attribute. 
This makes it easy to create composites that don't require callers to know about their components.  For a more detailed discussion of how this works as well as relevant context, we suggest reading the [delegated composition section of this blog article](https://www.fast.ai/2019/08/06/delegation/).

You can customise the behaviour of `GetAttr` in subclasses via;
- `_default`
    - By default, this is set to `'default'`, so attr access is passed down to `self.default`
    - `_default` can be set to the name of any instance attribute that does not start with dunder `__`
- `_xtra`
    - By default, this is `None`, so all attr access is passed down
    - You can limit which attrs get passed down by setting `_xtra` to a list of attribute names

To illuminate the utility of `GetAttr`, suppose we have the following two classes, `_WebPage` which is a superclass of `_ProductPage`, which we wish to compose like so:

In [ ]:
class _WebPage:
    def __init__(self, title, author="Jeremy"):
        self.title,self.author = title,author

class _ProductPage:
    def __init__(self, page, price): self.page,self.price = page,price
        
page = _WebPage('Soap', author="Sylvain")
p = _ProductPage(page, 15.0)

How do we make it so we can just write `p.author`, instead of `p.page.author` to access the `author` attribute?  We can use `GetAttr`, of course!  First, we subclass `GetAttr` when defining `_ProductPage`.  Next, we set `self.default` to the object whose attributes we want to be able to access directly, which in this case is the `page` argument passed on initialization:

In [ ]:
class _ProductPage(GetAttr):
    def __init__(self, page, price): self.default,self.price = page,price #self.default allows you to access page directly.

p = _ProductPage(page, 15.0)

Now, we can access the `author` attribute directly from the instance:

In [ ]:
test_eq(p.author, 'Sylvain')

If you wish to store the object you are composing in an attribute other than `self.default`, you can set the class attribute `_data` as shown below.  This is useful in the case where you might have a name collision with `self.default`:

In [ ]:
class _C(GetAttr):
    _default = '_data' # use different component name; `self._data` rather than `self.default`
    def __init__(self,a): self._data = a
    def foo(self): noop

t = _C('Hi')
test_eq(t._data, 'Hi') 
test_fail(lambda: t.default) # we no longer have self.default
test_eq(t.lower(), 'hi')
test_eq(t.upper(), 'HI')
assert 'lower' in dir(t)
assert 'upper' in dir(t)

By default, all attributes and methods of the object you are composing are retained. In the below example, we compose a `str` object with the class `_C`. This allows us to directly call string methods on instances of class `_C`, such as `str.lower()` or `str.upper()`:

In [ ]:
class _C(GetAttr):
    # allow all attributes and methods to get passed to `self.default` (by leaving _xtra=None)
    def __init__(self,a): self.default = a
    def foo(self): noop

t = _C('Hi')
test_eq(t.lower(), 'hi')
test_eq(t.upper(), 'HI')
assert 'lower' in dir(t)
assert 'upper' in dir(t)

However, you can choose which attributes or methods to retain by defining a class attribute `_xtra`, which is a list of allowed attribute and method names to delegate.  In the below example, we only delegate the `lower` method from the composed `str` object when defining class `_C`:

In [ ]:
class _C(GetAttr):
    _xtra = ['lower'] # specify which attributes get passed to `self.default`
    def __init__(self,a): self.default = a
    def foo(self): noop

t = _C('Hi')
test_eq(t.default, 'Hi')
test_eq(t.lower(), 'hi')
test_fail(lambda: t.upper()) # upper wasn't in _xtra, so it isn't available to be called
assert 'lower' in dir(t)
assert 'upper' not in dir(t)

You must be careful to properly set an instance attribute in `__init__` that corresponds to the class attribute `_default`.  The below example sets the class attribute `_default` to `data`, but erroneously fails to define `self.data` (and instead defines `self.default`).

Failing to properly set instance attributes leads to errors when you try to access methods directly:

In [ ]:
class _C(GetAttr):
    _default = 'data' # use a bad component name; i.e. self.data does not exist
    def __init__(self,a): self.default = a
    def foo(self): noop
        
# TODO: should we raise an error when we create a new instance ...
t = _C('Hi')
test_eq(t.default, 'Hi')
# ... or is it enough for all GetAttr features to raise errors
test_fail(lambda: t.data)
test_fail(lambda: t.lower())
test_fail(lambda: t.upper())
test_fail(lambda: dir(t))

In [ ]:
#hide
# I don't think this test is essential to the docs but it probably makes sense to
# check that everything works when we set both _xtra and _default to non-default values
class _C(GetAttr):
    _xtra = ['lower', 'upper']
    _default = 'data'
    def __init__(self,a): self.data = a
    def foo(self): noop

t = _C('Hi')
test_eq(t.data, 'Hi')
test_eq(t.lower(), 'hi')
test_eq(t.upper(), 'HI')
assert 'lower' in dir(t)
assert 'upper' in dir(t)

In [ ]:
#hide
#  when consolidating the filter logic, I choose the previous logic from 
# __getattr__  k.startswith('__') rather than
# _dir         k.startswith('_'). 
class _C(GetAttr):
    def __init__(self): self.default = type('_D', (), {'_under': 1, '__dunder': 2})() 
    
t = _C()
test_eq(t.default._under, 1)
test_eq(t._under, 1)           # _ prefix attr access is allowed on component
assert '_under' in dir(t)

test_eq(t.default.__dunder, 2)
test_fail(lambda: t.__dunder)  # __ prefix attr access is not allowed on component
assert '__dunder' not in dir(t)

assert t.__dir__ is not None   # __ prefix attr access is allowed on composite
assert '__dir__' in dir(t)

In [ ]:
#hide
#Failing test. TODO: make GetAttr pickle-safe

# class B:
#     def __init__(self): self.a = A()

# @funcs_kwargs
# class A(GetAttr):
#     wif=after_iter= noops
#     _methods = 'wif after_iter'.split()
#     _default = 'dataset'
#     def __init__(self, **kwargs): pass
    
# a = A()
# b = A(wif=a.wif)
        
# a = A()
# b = A(wif=a.wif)
# tst = pickle.dumps(b)
# c = pickle.loads(tst)

In [ ]:
#export
def delegate_attr(self, k, to):
    "Use in `__getattr__` to delegate to attr `to` without inheriting from `GetAttr`"
    if k.startswith('_') or k==to: raise AttributeError(k)
    try: return getattr(getattr(self,to), k)
    except AttributeError: raise AttributeError(k) from None

`delegate_attr` is a functional way to delegate attributes, and is an alternative to `GetAttr`.  We recommend reading the documentation of `GetAttr` for more details around delegation.

You can use achieve delegation when you define `__getattr__` by using `delegate_attr`:

In [ ]:
#hide
import pandas as pd

In [ ]:
class _C:
    def __init__(self, o): self.o = o # self.o corresponds to the `to` argument in delegate_attr.
    def __getattr__(self, k): return delegate_attr(self, k, to='o')
    

t = _C('HELLO') # delegates to a string
test_eq(t.lower(), 'hello')

t = _C(np.array([5,4,3])) # delegates to a numpy array
test_eq(t.sum(), 12)

t = _C(pd.DataFrame({'a': [1,2], 'b': [3,4]})) # delegates to a pandas.DataFrame
test_eq(t.b.max(), 4)

## Foundational Classes

In [ ]:
#export
class bind:
    "Same as `partial`, except you can use `arg0` `arg1` etc param placeholders"
    def __init__(self, fn, *pargs, **pkwargs):
        self.fn,self.pargs,self.pkwargs = fn,pargs,pkwargs
        self.maxi = max((x.i for x in pargs if isinstance(x, _Arg)), default=-1)

    def __call__(self, *args, **kwargs):
        args = list(args)
        kwargs = {**self.pkwargs,**kwargs}
        for k,v in kwargs.items():
            if isinstance(v,_Arg): kwargs[k] = args.pop(v.i)
        fargs = [args[x.i] if isinstance(x, _Arg) else x for x in self.pargs] + args[self.maxi+1:]
        return self.fn(*fargs, **kwargs)

In [ ]:
show_doc(bind, title_level=3)

<h3 id="bind" class="doc_header"><code>class</code> <code>bind</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>bind</code>(**`fn`**, **\*`pargs`**, **\*\*`pkwargs`**)

Same as `partial`, except you can use [`arg0`](/foundation.html#arg0) [`arg1`](/foundation.html#arg1) etc param placeholders

`bind` is the same as `partial`, but also allows you to reorder positional arguments using variable name(s) `arg{i}` where i refers to the zero-indexed positional argument. `bind` as implemented currently only supports reordering of up to the first 5 positional arguments.

Consider the function `myfunc` below, which has 3 positional arguments.  These arguments can be referenced as `arg0`, `arg1`, and `arg1`, respectively.  

In [ ]:
def myfn(a,b,c,d=1,e=2): return(a,b,c,d,e)

In the below example we bind the positional arguments of `myfn` as follows:

- The second input `14`, referenced by `arg1`, is substituted for the first positional argument.
- We supply a default value of `17` for the second positional argument.
- The first input `19`, referenced by `arg0`, is subsituted for the third positional argument.  

In [ ]:
test_eq(bind(myfn, arg1, 17, arg0, e=3)(19,14), (14,17,19,1,3))

In this next example:

- We set the default value to `17` for the first positional argument.
- The first input `19` refrenced by `arg0`, becomes the second positional argument.
- The second input `14` becomes the third positional argument.
- We override the default the value for named argument `e` to `3`.

In [ ]:
test_eq(bind(myfn, 17, arg0, e=3)(19,14), (17,19,14,1,3))

This is an example of using `bind` like `partial` and do not reorder any arguments:

In [ ]:
test_eq(bind(myfn)(17,19,14), (17,19,14,1,2))

`bind` can also be used to change default values.  In the below example, we use the first input `3` to override the default value of the named argument `e`, and supply default values for the first three positional arguments:

In [ ]:
test_eq(bind(myfn, 17,19,14,e=arg0)(3), (17,19,14,1,3))

## L -

In [ ]:
#export
listable_types = typing.Collection,Generator,map,filter,zip

In [ ]:
#export
class CollBase:
    "Base class for composing a list of `items`"
    def __init__(self, items): self.items = items
    def __len__(self): return len(self.items)
    def __getitem__(self, k): return self.items[list(k) if isinstance(k,CollBase) else k]
    def __setitem__(self, k, v): self.items[list(k) if isinstance(k,CollBase) else k] = v
    def __delitem__(self, i): del(self.items[i])
    def __repr__(self): return self.items.__repr__()
    def __iter__(self): return self.items.__iter__()

In [ ]:
show_doc(CollBase, title_level=3)

<h3 id="CollBase" class="doc_header"><code>class</code> <code>CollBase</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>CollBase</code>(**`items`**)

Base class for composing a list of `items`

In [ ]:
#export
class L(CollBase, metaclass=NewChkMeta):
    "Behaves like a list of `items` but can also index with list of indices or masks"
    _default='items'
    def __init__(self, items=None, *rest, use_list=False, match=None):
        if rest: items = (items,)+rest
        if items is None: items = []
        if (use_list is not None) or not _is_array(items):
            items = list(items) if use_list else _listify(items)
        if match is not None:
            if is_coll(match): match = len(match)
            if len(items)==1: items = items*match
            else: assert len(items)==match, 'Match length mismatch'
        super().__init__(items)

    @property
    def _xtra(self): return None
    def _new(self, items, *args, **kwargs): return type(self)(items, *args, use_list=None, **kwargs)
    def __getitem__(self, idx): return self._get(idx) if is_indexer(idx) else L(self._get(idx), use_list=None)
    def copy(self): return self._new(self.items.copy())

    def _get(self, i):
        if is_indexer(i) or isinstance(i,slice): return getattr(self.items,'iloc',self.items)[i]
        i = mask2idxs(i)
        return (self.items.iloc[list(i)] if hasattr(self.items,'iloc')
                else self.items.__array__()[(i,)] if hasattr(self.items,'__array__')
                else [self.items[i_] for i_ in i])

    def __setitem__(self, idx, o):
        "Set `idx` (can be list of indices, or mask, or int) items to `o` (which is broadcast if not iterable)"
        if isinstance(idx, int): self.items[idx] = o
        else:
            idx = idx if isinstance(idx,L) else _listify(idx)
            if not is_iter(o): o = [o]*len(idx)
            for i,o_ in zip(idx,o): self.items[i] = o_

    def __iter__(self): return iter(self.items.itertuples() if hasattr(self.items,'iloc') else self.items)
    def __contains__(self,b): return b in self.items
    def __reversed__(self): return self._new(reversed(self.items))
    def __invert__(self): return self._new(not i for i in self)
    def __eq__(self,b): return False if isinstance(b, (str,dict,set)) else all_equal(b,self)
    def __repr__(self): return repr(self.items) if _is_array(self.items) else coll_repr(self)
    def __mul__ (a,b): return a._new(a.items*b)
    def __add__ (a,b): return a._new(a.items+_listify(b))
    def __radd__(a,b): return a._new(b)+a
    def __addi__(a,b):
        a.items += list(b)
        return a

    def sorted(self, key=None, reverse=False):
        if isinstance(key,str):   k=lambda o:getattr(o,key,0)
        elif isinstance(key,int): k=itemgetter(key)
        else: k=key
        return self._new(sorted(self.items, key=k, reverse=reverse))

    @classmethod
    def split(cls, s, sep=None, maxsplit=-1): return cls(s.split(sep,maxsplit))

    @classmethod
    def range(cls, a, b=None, step=None):
        if is_coll(a): a = len(a)
        return cls(range(a,b,step) if step is not None else range(a,b) if b is not None else range(a))

    def map(self, f, *args, **kwargs):
        g = (bind(f,*args,**kwargs) if callable(f)
             else f.format if isinstance(f,str)
             else f.__getitem__)
        return self._new(map(g, self))

    def filter(self, f, negate=False, **kwargs):
        if kwargs: f = partial(f,**kwargs)
        if negate: f = negate_func(f)
        return self._new(filter(f, self))

    def argwhere(self, f, negate=False, **kwargs):
        if kwargs: f = partial(f,**kwargs)
        if negate: f = negate_func(f)
        return self._new(i for i,o in enumerate(self) if f(o))

    def unique(self): return L(dict.fromkeys(self).keys())
    def enumerate(self): return L(enumerate(self))
    def val2idx(self): return {v:k for k,v in self.enumerate()}
    def itemgot(self, *idxs):
        x = self
        for idx in idxs: x = x.map(itemgetter(idx))
        return x

    def attrgot(self, k, default=None): return self.map(lambda o: o.get(k,default) if isinstance(o, dict) else getattr(o,k,default))
    def cycle(self): return cycle(self)
    def map_dict(self, f=noop, *args, **kwargs): return {k:f(k, *args,**kwargs) for k in self}
    def starmap(self, f, *args, **kwargs): return self._new(itertools.starmap(partial(f,*args,**kwargs), self))
    def zip(self, cycled=False): return self._new((zip_cycle if cycled else zip)(*self))
    def zipwith(self, *rest, cycled=False): return self._new([self, *rest]).zip(cycled=cycled)
    def map_zip(self, f, *args, cycled=False, **kwargs): return self.zip(cycled=cycled).starmap(f, *args, **kwargs)
    def map_zipwith(self, f, *rest, cycled=False, **kwargs): return self.zipwith(*rest, cycled=cycled).starmap(f, **kwargs)
    def concat(self): return self._new(itertools.chain.from_iterable(self.map(L)))
    def shuffle(self):
        it = copy(self.items)
        random.shuffle(it)
        return self._new(it)

    def append(self,o): return self.items.append(o)
    def remove(self,o): return self.items.remove(o)
    def count (self,o): return self.items.count(o)
    def reverse(self ): return self.items.reverse()
    def pop(self,o=-1): return self.items.pop(o)
    def clear(self   ): return self.items.clear()
    def index(self, value, start=0, stop=sys.maxsize): return self.items.index(value, start, stop)
    def sort(self, key=None, reverse=False): return self.items.sort(key=key, reverse=reverse)
    def reduce(self, f, initial=None): return reduce(f, self) if initial is None else reduce(f, self, initial)
    def sum(self): return self.reduce(operator.add)
    def product(self): return self.reduce(operator.mul)

In [ ]:
#export
_docs = {o:"Passthru to `list` method" for o in
         'append count remove reverse sort pop clear index'.split()}
add_docs(L,
         __getitem__="Retrieve `idx` (can be list of indices, or mask, or int) items",
         range="Class Method: Same as `range`, but returns an `L`. Can pass a collection for `a`, to use `len(a)`",
         split="Class Method: Same as `str.split`, but returns an `L`",
         copy="Same as `list.copy`, but returns an `L`",
         sorted="New `L` sorted by `key`. If key is str then use `attrgetter`. If key is int then use `itemgetter`",
         unique="Unique items, in stable order",
         val2idx="Dict from value to index",
         filter="Create new `L` filtered by predicate `f`, passing `args` and `kwargs` to `f`",
         argwhere="Like `filter`, but return indices for matching items",
         map="Create new `L` with `f` applied to all `items`, passing `args` and `kwargs` to `f`",
         map_dict="Like `map`, but creates a dict from `items` to function results",
         starmap="Like `map`, but use `itertools.starmap`",
         itemgot="Create new `L` with item `idx` of all `items`",
         attrgot="Create new `L` with attr `k` of all `items`, if `items` contains dicts, then `L` will contain corresponding values for key `k` for each dict.",
         cycle="Same as `itertools.cycle`",
         enumerate="Same as `enumerate`",
         zip="Create new `L` with `zip(*items)`",
         zipwith="Create new `L` with `self` zip with each of `*rest`",
         map_zip="Combine `zip` and `starmap`",
         map_zipwith="Combine `zipwith` and `starmap`",
         concat="Concatenate all elements of list",
         shuffle="Same as `random.shuffle`, but not inplace",
         reduce="Wrapper for `functools.reduce`",
         sum="Sum of the items",
         product="Product of the items",
         **_docs)

In [ ]:
#export
Sequence.register(L);

You can create an `L` from an existing iterable (e.g. a list, range, etc) and access or modify it with an int list/tuple index, mask, int, or slice. All `list` methods can also be used with `L`.

In [ ]:
t = L(range(12))
test_eq(t, list(range(12)))
test_ne(t, list(range(11)))
t.reverse()
test_eq(t[0], 11)
t[3] = "h"
test_eq(t[3], "h")
t[3,5] = ("j","k")
test_eq(t[3,5], ["j","k"])
test_eq(t, L(t))
test_eq(L(L(1,2),[3,4]), ([1,2],[3,4]))
t

(#12) [11,10,9,'j',7,'k',5,4,3,2...]

Any `L` is a `Sequence` so you can use it with methods like `random.sample`:

In [ ]:
assert isinstance(t, Sequence)

In [ ]:
import random

In [ ]:
random.sample(t, 3)

[9, 7, 4]

In [ ]:
#hide
# test set items with L of collections
x = L([[1,2,3], [4,5], [6,7]])
x[0] = [1,2]
test_eq(x, L([[1,2], [4,5], [6,7]]))

There are optimized indexers for arrays, tensors, and DataFrames.

In [ ]:
arr = np.arange(9).reshape(3,3)
t = L(arr, use_list=None)
test_eq(t[1,2], arr[[1,2]])

arr = np.arange(9).reshape(3,3)
t = L(arr, use_list=None)
test_eq(t[1,2], arr[[1,2]])

df = pd.DataFrame({'a':[1,2,3]})
t = L(df, use_list=None)
test_eq(t[1,2], L(pd.DataFrame({'a':[2,3]}, index=[1,2]), use_list=None))

You can also modify an `L` with `append`, `+`, and `*`.

In [ ]:
t = L()
test_eq(t, [])
t.append(1)
test_eq(t, [1])
t += [3,2]
test_eq(t, [1,3,2])
t = t + [4]
test_eq(t, [1,3,2,4])
t = 5 + t
test_eq(t, [5,1,3,2,4])
test_eq(L(1,2,3), [1,2,3])
test_eq(L(1,2,3), L(1,2,3))
t = L(1)*5
t = t.map(operator.neg)
test_eq(t,[-1]*5)
test_eq(~L([True,False,False]), L([False,True,True]))
t = L(range(4))
test_eq(zip(t, L(1).cycle()), zip(range(4),(1,1,1,1)))
t = L.range(100)
test_shuffled(t,t.shuffle())

In [ ]:
def _f(x,a=0): return x+a
t = L(1)*5
test_eq(t.map(_f), t)
test_eq(t.map(_f,1), [2]*5)
test_eq(t.map(_f,a=2), [3]*5)

An `L` can be constructed from anything iterable, although tensors and arrays will not be iterated over on construction, unless you pass `use_list` to the constructor.

In [ ]:
test_eq(L([1,2,3]),[1,2,3])
test_eq(L(L([1,2,3])),[1,2,3])
test_ne(L([1,2,3]),[1,2,])
test_eq(L('abc'),['abc'])
test_eq(L(range(0,3)),[0,1,2])
test_eq(L(o for o in range(0,3)),[0,1,2])
test_eq(L(array(0)),[array(0)])
test_eq(L([array(0),array(1)]),[array(0),array(1)])
test_eq(L(array([0.,1.1]))[0],array([0.,1.1]))
test_eq(L(array([0.,1.1]), use_list=True), [array(0.),array(1.1)])  # `use_list=True` to unwrap arrays/arrays

If `match` is not `None` then the created list is same len as `match`, either by:

- If `len(items)==1` then `items` is replicated,
- Otherwise an error is raised if `match` and `items` are not already the same size.

In [ ]:
test_eq(L(1,match=[1,2,3]),[1,1,1])
test_eq(L([1,2],match=[2,3]),[1,2])
test_fail(lambda: L([1,2],match=[1,2,3]))

If you create an `L` from an existing `L` then you'll get back the original object (since `L` uses the `NewChkMeta` metaclass).

In [ ]:
test_is(L(t), t)

An `L` is considred equal to a list if they have the same elements. It's never considered equal to a `str` a `set` or a `dict` even if they have the same elements/keys.

In [ ]:
test_eq(L(['a', 'b']), ['a', 'b'])
test_ne(L(['a', 'b']), 'ab')
test_ne(L(['a', 'b']), {'a', 'b'})
test_ne(L(['a', 'b']), {'a':1, 'b':2})

### Class L Methods

In [ ]:
show_doc(L.__getitem__)

<h4 id="L.__getitem__" class="doc_header"><code>L.__getitem__</code><a href="__main__.py#L19" class="source_link" style="float:right">[source]</a></h4>

> <code>L.__getitem__</code>(**`idx`**)

Retrieve `idx` (can be list of indices, or mask, or int) items

In [ ]:
t = L(range(12))
test_eq(t[1,2], [1,2])                # implicit tuple
test_eq(t[[1,2]], [1,2])              # list
test_eq(t[:3], [0,1,2])               # slice
test_eq(t[[False]*11 + [True]], [11]) # mask
test_eq(t[array(3)], 3)

In [ ]:
show_doc(L.__setitem__)

<h4 id="L.__setitem__" class="doc_header"><code>L.__setitem__</code><a href="__main__.py#L29" class="source_link" style="float:right">[source]</a></h4>

> <code>L.__setitem__</code>(**`idx`**, **`o`**)

Set `idx` (can be list of indices, or mask, or int) items to `o` (which is broadcast if not iterable)

In [ ]:
t[4,6] = 0
test_eq(t[4,6], [0,0])
t[4,6] = [1,2]
test_eq(t[4,6], [1,2])

In [ ]:
show_doc(L.unique)

<h4 id="L.unique" class="doc_header"><code>L.unique</code><a href="__main__.py#L80" class="source_link" style="float:right">[source]</a></h4>

> <code>L.unique</code>()

Unique items, in stable order

In [ ]:
test_eq(L(1,2,3,4,4).unique(), [1,2,3,4])

In [ ]:
show_doc(L.val2idx)

<h4 id="L.val2idx" class="doc_header"><code>L.val2idx</code><a href="__main__.py#L82" class="source_link" style="float:right">[source]</a></h4>

> <code>L.val2idx</code>()

Dict from value to index

In [ ]:
test_eq(L(1,2,3).val2idx(), {3:2,1:0,2:1})

In [ ]:
show_doc(L.filter)

<h4 id="L.filter" class="doc_header"><code>L.filter</code><a href="__main__.py#L70" class="source_link" style="float:right">[source]</a></h4>

> <code>L.filter</code>(**`f`**, **`negate`**=*`False`*, **\*\*`kwargs`**)

Create new [`L`](/foundation.html#L) filtered by predicate `f`, passing `args` and `kwargs` to `f`

In [ ]:
list(t)

[0, 1, 2, 3, 1, 5, 2, 7, 8, 9, 10, 11]

In [ ]:
test_eq(t.filter(lambda o:o<5), [0,1,2,3,1,2])
test_eq(t.filter(lambda o:o<5, negate=True), [5,7,8,9,10,11])

In [ ]:
show_doc(L.argwhere)

<h4 id="L.argwhere" class="doc_header"><code>L.argwhere</code><a href="__main__.py#L75" class="source_link" style="float:right">[source]</a></h4>

> <code>L.argwhere</code>(**`f`**, **`negate`**=*`False`*, **\*\*`kwargs`**)

Like `filter`, but return indices for matching items

In [ ]:
test_eq(t.argwhere(lambda o:o<5), [0,1,2,3,4,6])

In [ ]:
show_doc(L.map)

<h4 id="L.map" class="doc_header"><code>L.map</code><a href="__main__.py#L64" class="source_link" style="float:right">[source]</a></h4>

> <code>L.map</code>(**`f`**, **\*`args`**, **\*\*`kwargs`**)

Create new [`L`](/foundation.html#L) with `f` applied to all `items`, passing `args` and `kwargs` to `f`

In [ ]:
test_eq(L.range(4).map(operator.neg), [0,-1,-2,-3])

If `f` is a string then it is treated as a format string to create the mapping:

In [ ]:
test_eq(L.range(4).map('#{}#'), ['#0#','#1#','#2#','#3#'])

If `f` is a dictionary (or anything supporting `__getitem__`) then it is indexed to create the mapping:

In [ ]:
test_eq(L.range(4).map(list('abcd')), list('abcd'))

If the special argument `_arg` is passed, then that is the kwarg used in the map.

In [ ]:
#What is this? TODO Jeremy: fix
#L.range(4).map(f, b=arg0)

In [ ]:
def f(a=None,b=None): return b
test_eq(L.range(4).map(f, b=arg0), range(4))

In [ ]:
show_doc(L.map_dict)

<h4 id="L.map_dict" class="doc_header"><code>L.map_dict</code><a href="__main__.py#L90" class="source_link" style="float:right">[source]</a></h4>

> <code>L.map_dict</code>(**`f`**=*`noop`*, **\*`args`**, **\*\*`kwargs`**)

Like `map`, but creates a dict from `items` to function results

In [ ]:
test_eq(L(range(1,5)).map_dict(), {1:1, 2:2, 3:3, 4:4})
test_eq(L(range(1,5)).map_dict(operator.neg), {1:-1, 2:-2, 3:-3, 4:-4})

In [ ]:
show_doc(L.zip)

<h4 id="L.zip" class="doc_header"><code>L.zip</code><a href="__main__.py#L92" class="source_link" style="float:right">[source]</a></h4>

> <code>L.zip</code>(**`cycled`**=*`False`*)

Create new [`L`](/foundation.html#L) with `zip(*items)`

In [ ]:
t = L([[1,2,3],'abc'])
test_eq(t.zip(), [(1, 'a'),(2, 'b'),(3, 'c')])

In [ ]:
t = L([[1,2,3,4],['a','b','c']])
test_eq(t.zip(cycled=True ), [(1, 'a'),(2, 'b'),(3, 'c'),(4, 'a')])
test_eq(t.zip(cycled=False), [(1, 'a'),(2, 'b'),(3, 'c')])

In [ ]:
show_doc(L.map_zip)

<h4 id="L.map_zip" class="doc_header"><code>L.map_zip</code><a href="__main__.py#L94" class="source_link" style="float:right">[source]</a></h4>

> <code>L.map_zip</code>(**`f`**, **\*`args`**, **`cycled`**=*`False`*, **\*\*`kwargs`**)

Combine `zip` and `starmap`

In [ ]:
t = L([1,2,3],[2,3,4])
test_eq(t.map_zip(operator.mul), [2,6,12])

In [ ]:
show_doc(L.zipwith)

<h4 id="L.zipwith" class="doc_header"><code>L.zipwith</code><a href="__main__.py#L93" class="source_link" style="float:right">[source]</a></h4>

> <code>L.zipwith</code>(**\*`rest`**, **`cycled`**=*`False`*)

Create new [`L`](/foundation.html#L) with `self` zip with each of `*rest`

In [ ]:
b = [[0],[1],[2,2]]
t = L([1,2,3]).zipwith(b)
test_eq(t, [(1,[0]), (2,[1]), (3,[2,2])])

In [ ]:
show_doc(L.map_zipwith)

<h4 id="L.map_zipwith" class="doc_header"><code>L.map_zipwith</code><a href="__main__.py#L95" class="source_link" style="float:right">[source]</a></h4>

> <code>L.map_zipwith</code>(**`f`**, **\*`rest`**, **`cycled`**=*`False`*, **\*\*`kwargs`**)

Combine `zipwith` and `starmap`

In [ ]:
test_eq(L(1,2,3).map_zipwith(operator.mul, [2,3,4]), [2,6,12])

In [ ]:
show_doc(L.itemgot)

<h4 id="L.itemgot" class="doc_header"><code>L.itemgot</code><a href="__main__.py#L83" class="source_link" style="float:right">[source]</a></h4>

> <code>L.itemgot</code>(**\*`idxs`**)

Create new [`L`](/foundation.html#L) with item `idx` of all `items`

In [ ]:
test_eq(t.itemgot(1), b)

In [ ]:
show_doc(L.attrgot)

<h4 id="L.attrgot" class="doc_header"><code>L.attrgot</code><a href="__main__.py#L88" class="source_link" style="float:right">[source]</a></h4>

> <code>L.attrgot</code>(**`k`**, **`default`**=*`None`*)

Create new [`L`](/foundation.html#L) with attr `k` of all `items`, if `items` contains dicts, then [`L`](/foundation.html#L) will contain corresponding values for key `k` for each dict.

In [ ]:
# Example when items are not a dict
a = [SimpleNamespace(a=3,b=4),SimpleNamespace(a=1,b=2)]
test_eq(L(a).attrgot('b'), [4,2])

#Example of when items are a dict
b =[{'id': 15, 'name': 'nbdev'}, {'id': 17, 'name': 'fastcore'}]
test_eq(L(b).attrgot('id'), [15, 17])

In [ ]:
show_doc(L.sorted)

<h4 id="L.sorted" class="doc_header"><code>L.sorted</code><a href="__main__.py#L50" class="source_link" style="float:right">[source]</a></h4>

> <code>L.sorted</code>(**`key`**=*`None`*, **`reverse`**=*`False`*)

New [`L`](/foundation.html#L) sorted by `key`. If key is str then use `attrgetter`. If key is int then use `itemgetter`

In [ ]:
test_eq(L(a).sorted('a').attrgot('b'), [2,4])

In [ ]:
show_doc(L.split)

<h4 id="L.split" class="doc_header"><code>L.split</code><a href="__main__.py#L56" class="source_link" style="float:right">[source]</a></h4>

> <code>L.split</code>(**`s`**, **`sep`**=*`None`*, **`maxsplit`**=*`-1`*)

Class Method: Same as `str.split`, but returns an [`L`](/foundation.html#L)

In [ ]:
test_eq(L.split('a b c'), list('abc'))

In [ ]:
show_doc(L.range)

<h4 id="L.range" class="doc_header"><code>L.range</code><a href="__main__.py#L59" class="source_link" style="float:right">[source]</a></h4>

> <code>L.range</code>(**`a`**, **`b`**=*`None`*, **`step`**=*`None`*)

Class Method: Same as `range`, but returns an [`L`](/foundation.html#L). Can pass a collection for `a`, to use `len(a)`

In [ ]:
test_eq_type(L.range([1,1,1]), L(range(3)))
test_eq_type(L.range(5,2,2), L(range(5,2,2)))

In [ ]:
show_doc(L.concat)

<h4 id="L.concat" class="doc_header"><code>L.concat</code><a href="__main__.py#L96" class="source_link" style="float:right">[source]</a></h4>

> <code>L.concat</code>()

Concatenate all elements of list

In [ ]:
test_eq(L([0,1,2,3],4,L(5,6)).concat(), range(7))

In [ ]:
show_doc(L.copy)

<h4 id="L.copy" class="doc_header"><code>L.copy</code><a href="__main__.py#L20" class="source_link" style="float:right">[source]</a></h4>

> <code>L.copy</code>()

Same as `list.copy`, but returns an [`L`](/foundation.html#L)

In [ ]:
t = L([0,1,2,3],4,L(5,6)).copy()
test_eq(t.concat(), range(7))

# Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_test.ipynb.
Converted 01_foundation.ipynb.
Converted 02_utils.ipynb.
Converted 03_dispatch.ipynb.
Converted 04_transform.ipynb.
Converted index.ipynb.
